In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

# Import Training Data and do some exploratory analysis

In [ ]:
#https://drive.google.com/file/d/1HJBqut9Ty5hUKnohAlWY-i-p4vz3BLSS/view?usp=sharing
!gdown --id 1HJBqut9Ty5hUKnohAlWY-i-p4vz3BLSS # train.csv

#https://drive.google.com/file/d/1hAZW6ZSboAMq073E0y6Uz1f_T2I12BGR/view?usp=sharing
!gdown --id 1hAZW6ZSboAMq073E0y6Uz1f_T2I12BGR # X.npy

#https://drive.google.com/file/d/11aderwsGMZvaYf_mgjuvbewTZRI2EWeY/view?usp=sharing
!gdown --id 11aderwsGMZvaYf_mgjuvbewTZRI2EWeY # y.npy

#https://drive.google.com/file/d/1QOv6Kby9CqHIt5CCshNBJutjExsFKu2L/view?usp=sharing
!gdown --id 1QOv6Kby9CqHIt5CCshNBJutjExsFKu2L # X_interpro&fill_nan_median

#https://drive.google.com/file/d/1DJ0RdlLGZR4OkSTNU1yL6zp5U9C-s_Tl/view?usp=sharing

!gdown --id 1DJ0RdlLGZR4OkSTNU1yL6zp5U9C-s_Tl # X_test_interpro&fill_nan_median



#https://drive.google.com/file/d/1F6H2xVEBobtUoqEKk5g2ST6QVN6oe-1p/view?usp=sharing
!gdown --id 1F6H2xVEBobtUoqEKk5g2ST6QVN6oe-1p #asset_id_corresponding_to_X_test

#https://drive.google.com/file/d/1tN5j9IvaUI_b1JCGrgY_Za9bLXHcm7oj/view?usp=sharing
!gdown --id 1tN5j9IvaUI_b1JCGrgY_Za9bLXHcm7oj #X_test_fully_scaled_preprocessed_with_time_index

#https://drive.google.com/file/d/1PPUtdyH7Xl9HxCwkuaK7znB1AKpZuEnC/view?usp=sharing
!gdown --id 1PPUtdyH7Xl9HxCwkuaK7znB1AKpZuEnC #X_train_fully_scaled_preprocessed_with_time_index

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1HJBqut9Ty5hUKnohAlWY-i-p4vz3BLSS
To: /content/train.csv
100% 33.1M/33.1M [00:00<00:00, 56.5MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1hAZW6ZSboAMq073E0y6Uz1f_T2I12BGR
To: /content/X.npy
100% 4.84M/4.84M [00:00<00:00, 21.1MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google

In [ ]:
X = np.load('X_train_fully_scaled_preprocessed_with_time_index.npy')
y = np.load('y.npy')

X = np.nan_to_num(X, nan=0.0)


assert not np.isnan(X).any(), "X contains NaN values"
assert not np.isnan(y).any(), "y contains NaN values"
assert not np.isinf(X).any(), "X contains Inf values"
assert not np.isinf(y).any(), "y contains Inf values"

X_test = np.load('X_test_fully_scaled_preprocessed_with_time_index.npy')

df_test = pd.read_csv('asset_id_corresponding_to_X_test.csv')

X_test = np.nan_to_num(X_test, nan=0.0)


assert not np.isnan(X_test).any(), "X contains NaN values"
assert not np.isinf(X_test).any(), "y contains Inf values"

In [ ]:
X.shape

(238, 229, 40)

In [ ]:
X.shape

(238, 229, 40)

In [ ]:
# Parameters
n_embd = X.shape[2] * X.shape[1]
T = X.shape[1]
C = X.shape[2]
max_seq_length = n_embd
n_layer = 8 #4
n_head = 4  #4
n_hidden = 1000
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
seed = 42

In [ ]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

class TimeSeriesDataset(Dataset):
    def __init__(self, X, y=None, augment=True):
        self.X = torch.tensor(X, dtype=torch.float64)
        self.y = torch.tensor(y, dtype=torch.float64) if y is not None else None
        self.augment = augment

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        x = self.X[idx]
        y = self.y[idx] if self.y is not None else None

        if self.augment and y is not None:
            x = self.augment_data(x)

        if y is not None:
            return x, y
        else:
            return x

    def augment_data(self, x):
        shift = np.random.randint(-5, 6)
        x = torch.roll(x, shifts=shift, dims=0)

        for i in range(x.shape[1]):
            shift = np.random.randint(-5, 6)
            x[:, i] = torch.roll(x[:, i], shifts=shift)

        noise = torch.normal(0, 0.01, x.shape)
        x += noise

        multiplier = torch.tensor(np.random.uniform(0.95, 1.05, x.shape[1]), dtype=torch.float64)
        x *= multiplier

        return x

# Create dataset
dataset = TimeSeriesDataset(X, y)

# Create DataLoader
dataloader = DataLoader(dataset, batch_size=8, shuffle=True)


In [ ]:

class HeadEncoder(nn.Module):
    def __init__(self, n_embd, head_size, T, C, window_size=300):
        super().__init__()
        self.n_embd = n_embd
        self.head_size = head_size
        self.T = T
        self.C = C
        self.window_size = window_size
        # Linear transformations for queries, keys, and values
        self.query = nn.Linear(T, head_size, bias=False, dtype=torch.float64)
        self.key = nn.Linear(T, head_size, bias=False, dtype=torch.float64)
        self.value = nn.Linear(T, head_size, bias=False, dtype=torch.float64)

    def forward(self, x):
        B, T = x.shape
        x = x.view(B, self.T, self.C)
        x = x.permute(0, 2, 1)  # Permute to (B, C, T)

        k = self.key(x)  # (B, C, head_size)
        q = self.query(x)  # (B, C, head_size)
        v = self.value(x)  # (B, C, head_size)

        k = k.permute(0, 2, 1)  # (B, head_size, C)
        q = q.permute(0, 2, 1)  # (B, head_size, C)

        # Attention scores
        attn = q @ k.transpose(-2, -1) * self.T**-0.5  # (B, C, head_size, head_size)

        # Apply random masking to 40% of the attention weights
        random_mask = torch.rand(attn.shape, device=attn.device, dtype=torch.float64) < 0.30
        attn[random_mask] = float('-inf')

        # Apply window-based masking
        for i in range(self.head_size):
            left = max(0, i - self.window_size)
            right = min(self.head_size, i + self.window_size + 1)

            # Mask out tokens outside the window
            if left > 0:
                attn[:, i, :left] = float('-inf')
            if right < T:
                attn[:, i, right:] = float('-inf')

        # Ensure no row is fully masked
        all_inf_mask = (attn == float('-inf')).all(dim=-1, keepdim=True)
        attn = torch.where(all_inf_mask, torch.zeros_like(attn), attn)

        attn = F.softmax(attn, dim=-1)
        out = v @ attn  # (B, C, head_size)
        return out
class MultiHeadAttentionEncoder(nn.Module):
    def __init__(self, n_embd, n_head, T, C):
        super().__init__()
        head_size = T // n_head
        self.heads = nn.ModuleList([HeadEncoder(n_embd, head_size, T, C) for _ in range(n_head)])
        self.proj = nn.Linear(head_size*n_head, T, dtype=torch.float64)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        B, T = x.shape
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        out = out.permute(0, 2, 1)
        out = out.contiguous().view(B, -1)
        return out

class FeedForwardNetworkEncoder(nn.Module):
    def __init__(self, n_embd):
        super().__init__()
        self.fc1 = nn.Linear(n_embd, n_embd, dtype=torch.float64)
        self.fc2 = nn.Linear(n_embd, n_embd, dtype=torch.float64)
        self.dropout = nn.Dropout(0.30)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

class ConvolutionalNetworkEncoder(nn.Module):
    def __init__(self, T, C):
        super().__init__()
        self.T = T
        self.C = C
        # 1D convolution layers with kernel size covering time steps
        self.conv1 = nn.Conv1d(in_channels=C, out_channels=C, kernel_size=5, padding=2, dtype=torch.float64)
        self.conv2 = nn.Conv1d(in_channels=C, out_channels=C, kernel_size=5, padding=2, dtype=torch.float64)
        self.conv3 = nn.Conv1d(in_channels=C, out_channels=C, kernel_size=5, padding=2, dtype=torch.float64)
        self.dropout = nn.Dropout(0.30)

    def forward(self, x):
        B, TC = x.shape
        assert TC == self.T * self.C, "Input dimensions do not match T and C"

        # Reshape to [B, T, C]
        x = x.view(B, self.T, self.C)

        # Permute to [B, C, T] for convolution
        x = x.permute(0, 2, 1)

        # Apply the first 1D convolutional layer
        x = F.relu(self.conv1(x))
        x = self.dropout(x)
        # Apply the second 1D convolutional layer
        x = F.relu(self.conv2(x))
        x = self.dropout(x)
        x = self.conv3(x)

        # Permute back to [B, T, C]
        x = x.permute(0, 2, 1)

        # Reshape back to [B, T*C]
        x = x.contiguous().view(B, -1)

        return x

class RegressionHead(nn.Module):
    def __init__(self, n_input, n_hidden):
        super().__init__()
        self.fc1 = nn.Linear(n_input, n_hidden * 2, dtype=torch.float64)
        self.fc2 = nn.Linear(n_hidden * 2, n_hidden, dtype=torch.float64)
        self.fc3 = nn.Linear(n_hidden, n_hidden // 2, dtype=torch.float64)
        self.fc4 = nn.Linear(n_hidden // 2, n_hidden // 4, dtype=torch.float64)
        self.fc6 = nn.Linear(n_hidden // 4, 1, dtype=torch.float64)
        self.dropout = nn.Dropout(0.2)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = self.fc6(x)
        return x

class LSTMHead(nn.Module):
    def __init__(self, n_embd, n_hidden, T, C):
        super().__init__()
        self.lstm = nn.LSTM(input_size=C, hidden_size=C, batch_first=True, dtype=torch.float64)
        self.C = C
        self.T = T

    def forward(self, x):
        B, TC = x.shape
        x = x.view(B, self.T, self.C)
        x, _ = self.lstm(x)
        x = x.contiguous().view(B, -1)

        return x

class EncoderBlock(nn.Module):
    def __init__(self, n_embd, n_head, T, C):
        super().__init__()
        self.self_attention = MultiHeadAttentionEncoder(n_embd, n_head, T, C)
        self.ffn = FeedForwardNetworkEncoder(n_embd)
        self.lstm = LSTMHead(n_embd, n_hidden, T, C)
        self.ln1 = nn.LayerNorm(n_embd, dtype=torch.float64)
        self.ln2 = nn.LayerNorm(n_embd, dtype=torch.float64)
        self.ln3 = nn.LayerNorm(n_embd, dtype=torch.float64)

    def forward(self, x):
        x = x + self.self_attention(self.ln1(x))
        x = x + self.ffn(self.ln2(x))
        x = x + self.lstm(self.ln3(x))
        return x

class EncoderRegression(nn.Module):
    def __init__(self, n_embd, n_layer, n_head, n_hidden, device, seed, max_seq_length, T, C):
        super().__init__()
        self.blocks = nn.Sequential(*[EncoderBlock(n_embd, n_head, T, C) for _ in range(n_layer)])
        #self.lstm = LSTMHead(n_embd, n_hidden, T, C)
        self.regression_head = RegressionHead(n_embd, n_hidden)
        self.ln_f = nn.LayerNorm(n_embd, dtype=torch.float64)
        self.positional_embeddings = nn.Parameter(torch.zeros(1, C, T, dtype=torch.float64))

        self.device = device

        torch.manual_seed(seed)
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, xbatch, ybatch=None):
        B, T, C = xbatch.shape

        # Reshape input to (B, C, T)
        xbatch = xbatch.permute(0, 2, 1)  # (B, T, C) -> (B, C, T)

        # Adding positional embeddings
        pos_emb = self.positional_embeddings[:, :, :T]  # (1, C, T)
        xbatch = xbatch + pos_emb

        # Reshape input to (B, T, C)
        xbatch = xbatch.permute(0, 2, 1)  # (B, C, T)
        xbatch = xbatch.reshape(xbatch.size(0), -1)

        # Process through encoder blocks
        input = self.blocks(xbatch)

        # Layer normalization
        input = self.ln_f(input)

        out = self.regression_head(input)

        # Compute loss if ybatch is provided
        loss = None
        if ybatch is not None:
            loss = F.l1_loss(out, ybatch.unsqueeze(1).double())

        return out, loss

In [ ]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, fold=None):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.fold = fold

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), f'checkpoint_fold_{self.fold}.pth')
        self.val_loss_min = val_loss



# Cross-validation parameters
n_splits_k_fold = 5
epochs = 2000#1000
patience = 10
seed = 42
learning_rate = 0.00005 #0.00005

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

kf = KFold(n_splits=n_splits_k_fold, shuffle=True, random_state=seed)
train_losses_per_fold = []
val_losses_per_fold = []

for fold, (train_idx, val_idx) in enumerate(kf.split(X)):
    print(f'Fold {fold+1}')
    X_train, X_val = X[train_idx], X[val_idx]
    y_train, y_val = y[train_idx], y[val_idx]

    train_dataset = TimeSeriesDataset(X_train, y_train)
    val_dataset = TimeSeriesDataset(X_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)

    model = EncoderRegression(n_embd=n_embd, n_layer=n_layer, n_head=n_head, n_hidden=n_hidden, device=device, seed=seed, max_seq_length=max_seq_length, T=T, C=C).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)#lr = 0.00001
    criterion = nn.L1Loss()

    early_stopping = EarlyStopping(patience=patience, verbose=True, fold=fold+1)

    train_losses = []
    val_losses = []

    for epoch in range(epochs):  # Number of epochs
        model.train()
        running_train_loss = 0.0
        for xbatch, ybatch in train_loader:
            xbatch, ybatch = xbatch.to(device), ybatch.to(device)

            optimizer.zero_grad()
            out, loss = model(xbatch, ybatch)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            running_train_loss += loss.item()

        avg_train_loss = running_train_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for xbatch, ybatch in val_loader:
                xbatch, ybatch = xbatch.to(device), ybatch.to(device)
                out, loss = model(xbatch, ybatch)
                running_val_loss += loss.item()

        avg_val_loss = running_val_loss / len(val_loader)
        val_losses.append(avg_val_loss)

        if epoch % 5 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Fold {fold+1}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')

        # Check early stopping
        early_stopping(avg_val_loss, model)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    train_losses_per_fold.append(train_losses)
    val_losses_per_fold.append(val_losses)

    # Load the last checkpoint with the best model for this fold
    model.load_state_dict(torch.load(f'checkpoint_fold_{fold+1}.pth'))

# Plotting learning curves for each fold
fig, axes = plt.subplots(n_splits_k_fold, 1, figsize=(10, 25), sharex=True)

for fold in range(n_splits_k_fold):
    axes[fold].plot(train_losses_per_fold[fold], label='Training Loss')
    axes[fold].plot(val_losses_per_fold[fold], label='Validation Loss')
    axes[fold].set_title(f'Fold {fold+1}')
    axes[fold].set_xlabel('Epoch')
    axes[fold].set_ylabel('Loss')
    axes[fold].legend()

plt.tight_layout()
plt.show()

In [ ]:
# Flatten the lists
train_losses = [item for sublist in train_losses_per_fold for item in sublist]
val_losses = [item for sublist in val_losses_per_fold for item in sublist]

# Determine fold boundaries
fold_sizes = [len(fold) for fold in train_losses_per_fold]
fold_boundaries = [sum(fold_sizes[:i+1]) for i in range(len(fold_sizes))]

# Plotting learning curves
plt.figure(figsize=(10, 6))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')

# Adding red dashed lines to separate each fold
for boundary in fold_boundaries:
    plt.axvline(x=boundary, color='red', linestyle='--', linewidth=1)

plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Learning Curves')
plt.legend()
plt.show()

# Import Test Data and Predicting it

In [ ]:
from google.colab import files
files.download('checkpoint_fold_5.pth')

In [ ]:
B, T, C = X_test.shape

print(T)

In [ ]:
test_dataset = TimeSeriesDataset(X_test)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# Placeholder for storing predictions from each fold
predictions_per_fold = []

for fold in range(n_splits_k_fold):
    # Initialize the model
    model = EncoderRegression(n_embd=n_embd, n_layer=n_layer, n_head=n_head, n_hidden=n_hidden, device=device, seed=seed, max_seq_length=max_seq_length, T=T, C=C).to(device)

    # Load the best model for the current fold
    model.load_state_dict(torch.load(f'checkpoint_fold_{fold+1}.pth'))
    model.eval()

    # Collect predictions
    fold_predictions = []
    with torch.no_grad():
        for xbatch in test_loader:
            xbatch = xbatch.to(device)
            out, _ = model(xbatch)  # Assuming model(xbatch) returns (output, loss)
            fold_predictions.append(out.cpu().numpy())

    # Concatenate predictions for the current fold
    fold_predictions = np.concatenate(fold_predictions, axis=0)
    predictions_per_fold.append(fold_predictions)

# Convert the list of predictions to a NumPy array for easier manipulation
predictions_per_fold = np.array(predictions_per_fold)

# Average the predictions from all folds




In [ ]:
#df_test = pd.read_csv('test.csv')
df_test = pd.read_csv('/content/asset_id_corresponding_to_X_test.csv')

In [ ]:
predictions = [i[0] for i in predictions_per_fold[3]]


submission_file=pd.DataFrame()
assets=pd.Series(df_test.asset_id.unique()).rename('asset_id')
predictions=pd.Series(predictions).rename('pediction')
submission_file=pd.concat([assets,predictions],axis=1)
submission_file.to_csv('Submission.csv', index=False)

In [ ]:
Ensemble_predictions = np.mean(predictions_per_fold[[0,1,3]], axis=0)

predictions = [i[0] for i in Ensemble_predictions]

submission_file=pd.DataFrame()
assets=pd.Series(df_test.asset_id.unique()).rename('asset_id')
predictions=pd.Series(predictions).rename('pediction')
submission_file=pd.concat([assets,predictions],axis=1)
submission_file.to_csv('Ensemble_predictions.csv', index=False)